In [3]:
import os

import geopandas as gpd
from pyspark.sql import SparkSession

from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [4]:
spark = SparkSession. \
    builder. \
    appName('appName'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,org.datasyslab:geotools-wrapper:geotools-24.0'). \
    getOrCreate()

In [5]:
SedonaRegistrator.registerAll(spark)

True

## Geometry Constructors

### ST_Point

In [6]:
point_csv_df = spark.read.format("csv").\
    option("delimiter", ",").\
    option("header", "false").\
    load("data/testpoint.csv")

point_csv_df.createOrReplaceTempView("pointtable")

point_df = spark.sql("select ST_Point(cast(pointtable._c0 as Decimal(24,20)), cast(pointtable._c1 as Decimal(24,20))) as arealandmark from pointtable")
point_df.show(5)

+-----------------+
|     arealandmark|
+-----------------+
|POINT (1.1 101.1)|
|POINT (2.1 102.1)|
|POINT (3.1 103.1)|
|POINT (4.1 104.1)|
|POINT (5.1 105.1)|
+-----------------+
only showing top 5 rows



### ST_GeomFromText

In [7]:
polygon_wkt_df = spark.read.format("csv").\
    option("delimiter", "\t").\
    option("header", "false").\
    load("data/county_small.tsv")

polygon_wkt_df.createOrReplaceTempView("polygontable")
polygon_df = spark.sql("select polygontable._c6 as name, ST_GeomFromText(polygontable._c0) as countyshape from polygontable")
polygon_df.show(5)

+----------------+--------------------+
|            name|         countyshape|
+----------------+--------------------+
|   Cuming County|POLYGON ((-97.019...|
|Wahkiakum County|POLYGON ((-123.43...|
|  De Baca County|POLYGON ((-104.56...|
|Lancaster County|POLYGON ((-96.910...|
| Nuckolls County|POLYGON ((-98.273...|
+----------------+--------------------+
only showing top 5 rows



### ST_GeomFromWKB

In [8]:
polygon_wkb_df = spark.read.format("csv").\
    option("delimiter", "\t").\
    option("header", "false").\
    load("data/county_small_wkb.tsv")

polygon_wkb_df.createOrReplaceTempView("polygontable")
polygon_df = spark.sql("select polygontable._c6 as name, ST_GeomFromWKB(polygontable._c0) as countyshape from polygontable")
polygon_df.show(5)

+----------------+--------------------+
|            name|         countyshape|
+----------------+--------------------+
|   Cuming County|POLYGON ((-97.019...|
|Wahkiakum County|POLYGON ((-123.43...|
|  De Baca County|POLYGON ((-104.56...|
|Lancaster County|POLYGON ((-96.910...|
| Nuckolls County|POLYGON ((-98.273...|
+----------------+--------------------+
only showing top 5 rows



### ST_GeomFromGeoJSON

In [9]:
polygon_json_df = spark.read.format("csv").\
    option("delimiter", "\t").\
    option("header", "false").\
    load("data/testPolygon.json")

polygon_json_df.createOrReplaceTempView("polygontable")
polygon_df = spark.sql("select ST_GeomFromGeoJSON(polygontable._c0) as countyshape from polygontable")
polygon_df.show(5)

+--------------------+
|         countyshape|
+--------------------+
|POLYGON ((-87.621...|
|POLYGON ((-85.719...|
|POLYGON ((-86.000...|
|POLYGON ((-86.574...|
|POLYGON ((-85.382...|
+--------------------+
only showing top 5 rows



## Spatial Operations

### Spatial Join - Distance Join

In [10]:
point_csv_df_1 = spark.read.format("csv").\
    option("delimiter", ",").\
    option("header", "false").load("data/testpoint.csv")

point_csv_df_1.createOrReplaceTempView("pointtable")

point_df1 = spark.sql("SELECT ST_Point(cast(pointtable._c0 as Decimal(24,20)),cast(pointtable._c1 as Decimal(24,20))) as pointshape1, \'abc\' as name1 from pointtable")
point_df1.createOrReplaceTempView("pointdf1")

point_csv_df2 = spark.read.format("csv").\
    option("delimiter", ",").\
    option("header", "false").load("data/testpoint.csv")

point_csv_df2.createOrReplaceTempView("pointtable")
point_df2 = spark.sql("select ST_Point(cast(pointtable._c0 as Decimal(24,20)),cast(pointtable._c1 as Decimal(24,20))) as pointshape2, \'def\' as name2 from pointtable")
point_df2.createOrReplaceTempView("pointdf2")

distance_join_df = spark.sql("select * from pointdf1, pointdf2 where ST_Distance(pointdf1.pointshape1,pointdf2.pointshape2) < 2")
distance_join_df.explain()
distance_join_df.show(5)

== Physical Plan ==
DistanceJoin pointshape1#261: geometry, pointshape2#287: geometry, 2.0, false
:- Project [st_point(cast(_c0#255 as decimal(24,20)), cast(_c1#256 as decimal(24,20))) AS pointshape1#261, abc AS name1#262]
:  +- FileScan csv [_c0#255,_c1#256] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/jiayu/GitHub/GeoSpark-datasys-repo/binder/data/testpoint.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string>
+- Project [st_point(cast(_c0#281 as decimal(24,20)), cast(_c1#282 as decimal(24,20))) AS pointshape2#287, def AS name2#288]
   +- FileScan csv [_c0#281,_c1#282] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/jiayu/GitHub/GeoSpark-datasys-repo/binder/data/testpoint.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string>


+-----------------+-----+-----------------+-----+
|      pointshape1|name1|      pointshape2|name2|
+-----------

### Spatial Join - Range Join and RDD API Join

Please refer to the example - airports per country: https://github.com/apache/incubator-sedona/blob/master/binder/ApacheSedonaSQL_SpatialJoin_AirportsPerCountry.ipynb

### Converting GeoPandas to Apache Sedona

In [9]:
gdf = gpd.read_file("data/gis_osm_pois_free_1.shp")

osm_points = spark.createDataFrame(
    gdf
)

In [10]:
osm_points.printSchema()

root
 |-- osm_id: string (nullable = true)
 |-- code: long (nullable = true)
 |-- fclass: string (nullable = true)
 |-- name: string (nullable = true)
 |-- geometry: geometry (nullable = true)



In [11]:
osm_points.show(5)

+--------+----+---------+--------------+--------------------+
|  osm_id|code|   fclass|          name|            geometry|
+--------+----+---------+--------------+--------------------+
|26860257|2422|camp_site|      de Kroon|POINT (15.3393145...|
|26860294|2406|   chalet|Leśne Ustronie|POINT (14.8709625...|
|29947493|2402|    motel|          null|POINT (15.0946636...|
|29947498|2602|      atm|          null|POINT (15.0732014...|
|29947499|2401|    hotel|          null|POINT (15.0696777...|
+--------+----+---------+--------------+--------------------+
only showing top 5 rows



In [12]:
osm_points.createOrReplaceTempView("points")

In [13]:
transformed_df = spark.sql(
    """
        SELECT osm_id,
               code,
               fclass,
               name,
               ST_Transform(geometry, 'epsg:4326', 'epsg:2180') as geom 
        FROM points
    """)

In [14]:
transformed_df.show(5)

+--------+----+---------+--------------+--------------------+
|  osm_id|code|   fclass|          name|                geom|
+--------+----+---------+--------------+--------------------+
|26860257|2422|camp_site|      de Kroon|POINT (-3288183.3...|
|26860294|2406|   chalet|Leśne Ustronie|POINT (-3341183.9...|
|29947493|2402|    motel|          null|POINT (-3320466.5...|
|29947498|2602|      atm|          null|POINT (-3323205.7...|
|29947499|2401|    hotel|          null|POINT (-3323655.1...|
+--------+----+---------+--------------+--------------------+
only showing top 5 rows



In [15]:
transformed_df.createOrReplaceTempView("points_2180")

In [16]:
neighbours_within_1000m = spark.sql("""
        SELECT a.osm_id AS id_1,
               b.osm_id AS id_2,
               a.geom 
        FROM points_2180 AS a, points_2180 AS b 
        WHERE ST_Distance(a.geom,b.geom) < 50
    """)

In [17]:
neighbours_within_1000m.show()

+---------+----------+--------------------+
|     id_1|      id_2|                geom|
+---------+----------+--------------------+
|197624402| 197624402|POINT (-3383818.5...|
|197663196| 197663196|POINT (-3383367.1...|
|197953474| 197953474|POINT (-3383763.3...|
|262310516| 262310516|POINT (-3384257.6...|
|262310516|1074233123|POINT (-3384257.6...|
|270281140| 270281140|POINT (-3385421.2...|
|270281140|1074232906|POINT (-3385421.2...|
|270306609| 270306609|POINT (-3383982.8...|
|270306746| 270306746|POINT (-3383898.4...|
|293896571| 293896571|POINT (-3385029.0...|
|293896571|3256728465|POINT (-3385029.0...|
|360178884| 360178884|POINT (-3377483.1...|
|360178897| 360178897|POINT (-3374350.0...|
|360178897| 360178897|POINT (-3374350.0...|
|360178897|5546280698|POINT (-3374350.0...|
|360178897|5546280699|POINT (-3374350.0...|
|360178897| 360178897|POINT (-3374350.0...|
|360178897| 360178897|POINT (-3374350.0...|
|360178897|5546280698|POINT (-3374350.0...|
|360178897|5546280699|POINT (-33

## Converting Apache Sedona to GeoPandas

In [18]:
df = neighbours_within_1000m.toPandas()

In [19]:
gdf = gpd.GeoDataFrame(df, geometry="geom")

In [20]:
gdf

,id_1,id_2,geom
0,197624402,197624402,POINT (-3383818.580 4179182.169)
1,197663196,197663196,POINT (-3383367.151 4179427.096)
2,197953474,197953474,POINT (-3383763.332 4179408.785)
3,262310516,262310516,POINT (-3384257.682 4178033.053)
4,262310516,1074233123,POINT (-3384257.682 4178033.053)
...,...,...,...
45314,6620325385,6620325385,POINT (-3215183.489 4307887.823)
45315,6631077531,6631077531,POINT (-3227737.389 4306566.622)
45316,6736467188,6736467188,POINT (-3242695.331 4298828.321)
45317,6736772185,6736772185,POINT (-3204857.139 4313763.361)
